In [1]:
#Installation de Selenium

#!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb -O /tmp/chrome.deb
#!sudo apt-get update
#!sudo -E apt-get install -y /tmp/chrome.deb
#!pip install chromedriver-autoinstaller selenium

In [2]:
#!pip install unidecode

In [3]:
import scrapping.scrapping_ciqual

data_tools importé !
conversion_tools importé !
scraping_marmiton importé ! 
scrapping_ciqual importé !


In [4]:
from importlib import reload
reload(scrapping.scrapping_ciqual)

scrapping_ciqual importé !


<module 'scrapping.scrapping_ciqual' from '/home/onyxia/work/Marmiton/scrapping/scrapping_ciqual.py'>